In [2]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm


# Text Preprocessing and NLP
import nltk

# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize

# Part-of-speech tagging
from nltk import pos_tag

# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer

In [1]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/dariusng2103/projects/dm_project/DM-Fake-News-Detection


### 1. Load train and test datasets, for both original and rewritten (using LLM)

In [4]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

# Basic BERT

BERT is a transformer model that provides powerful pre-trained embeddings for downstream tasks such as fake news classification using text.

### Transformer Architecture
To use BERT in Tensorflow, we utilise the `transformers` library by HuggingFace, which simplifies the process of loading pre-trained BERT models and tokenizers. The transformer uses a mechanism called 'self-attention' to weigh the importance of each word in a sentence relative to others, allowing it to process entire sentences at once instead of word-by-word.

### Bidirectional Context Understanding
BERT reads text in both directions at once which helps BERT to understand the full context of each word in relation to its surrounding words, making it excellent at capturing meaning, nuances, and relationships in language.

### Pretraining Tasks
- **Masked Language Modelling (MLM):** Randomly masks some words in sentences and trains the model to predict them. This task encourages BERT to learn contextual relationships and gain a better understanding of language.

- **Next Sentence Prediction (NSP):** Trains BERT to understand relationships between sentences by predicting whether one sentence naturally follows another. This helps BERT with tasks that require understanding sentence-pairs, like question-answering.

BERT is limited to a maximum input length of 512 tokens.

Fine-tuning BERT usually requires fewer epochs (2-4) and smaller batch sizes (16 or 32) due to memory constraints and pre-trained knowledge.



In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
import numpy as np
import random

# Set random seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

ModuleNotFoundError: No module named 'torch'